# Import

In [1]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import re
import contractions
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import collections
import math
import pickle


import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
STOPWORDS = set(stopwords.words("english"))

from torch.nn.utils.rnn import pad_sequence
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import Dataset, DataLoader, random_split 
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import pandas as p
import torch 

from tqdm import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print('device = ', DEVICE)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nguye\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


device =  cuda


# Constants 

In [2]:
ROOT = os.getcwd()
DATASET_PATH = os.path.join(ROOT, 'dataset', 'amazon_full' )
ALL_DF_CSV = os.path.join(DATASET_PATH, 'all_df.csv')
AMAZON_DATASET_SAVEPATH = os.path.join(DATASET_PATH, 'amazon_dataset.pickle')

OUTPUT_PATH = os.path.join(ROOT, 'output', 'bert')

if os.path.exists(OUTPUT_PATH) == False:
    os.makedirs(OUTPUT_PATH)

BATCH_SIZE = 64
SAMPLE_SIZE = 0.01
MODEL_NAME = 'prajjwal1/bert-mini'

# Load csv

In [3]:
all_df = pd.read_csv(ALL_DF_CSV)
all_df.drop('title', axis =1 , inplace=True)
NUM_CLASSES = len(all_df['review'].unique())
print("num class = ", NUM_CLASSES)
print(all_df.shape)
all_df.head()


num class =  5
(60000, 2)


,review,text
0,1,"Hm, I was definitely disappointed. This book i..."
1,2,It took me a while to fall in love with this a...
2,0,WORSTMOVIEEVER!Not a single laugh or other red...
3,0,"In the very first couple of lines, i knew some..."
4,3,COMBAT! THE BEST OF THE COLOR EPSODES is the o...


# Dataset

In [4]:
class AmazonReviewDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        review = self.df['text'].iloc[idx]
        label = self.df['review'].iloc[idx]
#         print('idx', idx)

        # Tokenize the review
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    labels = [item['labels'] for item in batch]

    input_ids = pad_sequence(input_ids, batch_first=True)
    attention_mask = pad_sequence(attention_mask, batch_first=True)
    labels = torch.tensor(labels)

    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

# BERT pretrained

In [5]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=NUM_CLASSES)

c:\Users\nguye\anaconda3\envs\torch\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-mini and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
all_dataset = AmazonReviewDataset(all_df, tokenizer)
print(len(all_df))

with open(AMAZON_DATASET_SAVEPATH, 'wb') as file:
    pickle.dump(all_dataset, file)

60000


In [7]:
# indices = [3332, 8107, 69566, 25697, 86807, 26598, 37591, 29156]

# # Iterate over each index and print the required information
# for idx in indices:
#     if idx in train_df.index:  # Check if the index exists in the DataFrame
#         text = train_df['text'][idx]
#         target = train_df['target'][idx]
#         text_length = len(text)
#         print(f"Index {idx}:")
#         print(f"Text: {text}")
#         print(f"Length of Text: {text_length}")
#         print(f"Target: {target}\n")
#     else:
#         print(f"Index {idx} not in DataFrame.\n")


# Create dataset and dataloader

In [8]:
total_train_len = len(all_df)
train_len = int(0.8 * total_train_len) 
val_len = int(0.2 * total_train_len) //2

train_dataset = AmazonReviewDataset(all_df, tokenizer)

train_dataset, val_dataset, test_dataset = random_split(train_dataset, [train_len, val_len, val_len])

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

print('train dataloader = ', len(train_dataloader))
print('valid dataloader = ', len(val_dataloader))
print('test dataloader = ', len(test_dataloader))

sample = next(iter(train_dataloader))
print('sample input ids = ', sample['input_ids'].shape)
print('sample attention mask = ', sample['attention_mask'].shape)
print('sample label shape = ', sample['labels'].shape)

train dataloader =  750
valid dataloader =  94
test dataloader =  94
sample input ids =  torch.Size([64, 512])
sample attention mask =  torch.Size([64, 512])
sample label shape =  torch.Size([64])


In [9]:
all_dataset = AmazonReviewDataset(all_df, tokenizer)
print(len(all_df))

with open(AMAZON_DATASET_SAVEPATH, 'wb') as file:
    pickle.dump(all_dataset, file)

60000


# Training utils

In [10]:
def get_accuracy(predictions, labels):
    return (predictions == labels).sum() / len(labels)

def train(dataloader, model, optimizer, device):
    model.train()
    epoch_losses = []
    epoch_accs = []

    for batch in tqdm(dataloader, desc="training..."):

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        output = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = output.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        logits = output.logits
        predictions = torch.argmax(logits, dim=1)
        accuracy = get_accuracy(predictions, labels)

        epoch_losses.append(loss.item())
        epoch_accs.append(accuracy.item())

    return np.mean(epoch_losses), np.mean(epoch_accs)

def evaluate(dataloader, model, device):
    model.eval()
    epoch_losses = []
    epoch_accs = []

    for batch in tqdm(dataloader, desc="evaluating..."):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        output = model(input_ids, attention_mask=attention_mask, labels=labels)

        logits = output.logits
        predictions = torch.argmax(logits, dim=1)
        accuracy = get_accuracy(predictions, labels)

        epoch_losses.append(output.loss.item())
        epoch_accs.append(accuracy.item())

            
    return np.mean(epoch_losses), np.mean(epoch_accs)

def plot(history, save_fig = None, show = False):
    fig, (ax1, ax2) = plt.subplots(1,2, figsize = (10,5))
    epochs = list(range(len(history['train_accs'])))

    sns.lineplot(y = history["train_accs"],   label ='train accuracy',  x = epochs, ax = ax1)
    sns.lineplot(y = history["valid_accs"],   label ='val accuracy',    x = epochs, ax = ax1)
    sns.lineplot(y = history["test_accs"],  label ='test accuracy', x = epochs, ax = ax1)
    ax1.set_title("Accuracy")

    sns.lineplot(y = history["train_losses"], label ='train loss', x = epochs, ax = ax2)
    sns.lineplot(y = history["valid_losses"],   label ='val loss', x = epochs, ax = ax2)
    sns.lineplot(y = history["test_losses"],  label ='test loss', x = epochs, ax = ax2)
    ax2.set_title("Loss")

    if save_fig:
        plt.savefig(save_fig)
    if show:
        plt.show()
    plt.clf()

def tune(model, optimizer, device, train_dataloader, val_dataloader, test_dataloader, epochs = 10, label = 'baseline'):
    SAVE_PATH = os.path.join(OUTPUT_PATH, label)
    PLOT_PATH = os.path.join(SAVE_PATH, 'plot.png')

    if os.path.exists(SAVE_PATH) == False:
        os.makedirs(SAVE_PATH)

    best_valid_loss = float("inf")
    history = collections.defaultdict(list)

    model.to(device)

    for epoch in range(epochs):
        train_loss, train_acc = train(train_dataloader, model, optimizer, device)
        val_loss, val_acc = evaluate(val_dataloader, model, device)
        test_loss, test_acc = evaluate(test_dataloader, model, device)

        history["train_losses"].append(train_loss)
        history["train_accs"].append(train_acc)
        history["valid_losses"].append(val_loss)
        history["valid_accs"].append(val_acc)
        history["test_losses"].append(test_loss)
        history["test_accs"].append(test_acc)

        if val_loss < best_valid_loss:
            best_valid_loss = val_loss 
            torch.save(model, os.path.join(SAVE_PATH, 'bert.checkpoint.torch'))

        print(f"epoch: {epoch}")
        print(f"train_loss: {train_loss:.3f}, train_acc: {train_acc:.3f}")
        print(f"val_loss: {val_loss:.3f}, valid_acc: {val_acc:.3f}")
        print(f"test_loss: {test_loss:.3f}, test_acc: {test_acc:.3f}")

        plot(history, save_fig=PLOT_PATH)
    
    plot(history, show=True)
    



# Tune

In [11]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device_name = torch.cuda.get_device_name(torch.cuda.current_device())
print("CUDA Device Name:", device_name)
training_epochs = 3

tune(model, optimizer, device, train_dataloader, val_dataloader, test_dataloader,
      epochs = training_epochs)

CUDA Device Name: NVIDIA GeForce RTX 3060


training...:  31%|███       | 233/750 [01:24<03:07,  2.76it/s]


KeyboardInterrupt: 